In [ ]:
!unzip -q /content/English-2000-Judgments.zip

In [ ]:
!pip3 install datasets==2.12.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 21.3 MB/s eta 0:00:00


In [ ]:
!pip3 install rouge-score -q

  Preparing metadata (setup.py) ... done


In [ ]:
test_file_names=['_judgment_judis_51.txt','_judgment_judis_97.txt','_judgment_judis_55.txt','_judgment_judis_11.txt','_judgment_judis_37.txt','_judgment_judis_66.txt','_judgment_judis_63.txt','_judgment_judis_53.txt','_judgment_judis_41.txt','_judgment_judis_62.txt']
val_file_names=['_judgment_judis_57.txt','_judgment_judis_60.txt','_judgment_judis_12.txt','_judgment_judis_39.txt','_judgment_judis_75.txt','_judgment_judis_72.txt','_judgment_judis_59.txt','_judgment_judis_44.txt','_judgment_judis_70.txt','_judgment_judis_50.txt']

In [ ]:
import os

judgement_files = os.listdir('/content/English-2000-Judgments')
print(len(judgement_files))
import re

def preprocess(text):
    # Extracting the text within the HEADNOTE section, if available
    headnote_missing = False
    headnote_pattern = re.compile(r'HEADNOTE:(.*)JUDGMENT\n:\n(.*)', re.DOTALL)
    headnote_match = headnote_pattern.search(text)
    if headnote_match is None:
        headnote_missing = True
    headnote_text = headnote_match.group(1)

    # Extracting the text within the JUDGMENT section, if available
    judgment_missing = False
    judgment_pattern = re.compile(r'HEADNOTE:(.*)JUDGMENT\n:\n(.*)', re.DOTALL)
    judgment_match = judgment_pattern.search(text)
    if judgment_match is None:
        judgment_missing = True
    judgment_text = judgment_match.group(2)

    if headnote_missing and judgment_missing:
        return "Headnote and Judgement missing"
    elif headnote_missing:
        return "Headnote missing"
    elif judgment_missing:
        return "Judgment missing"

    data = {}
    headnote_text = re.sub(r"\n", " ", headnote_text)
    while "  " in headnote_text:
        headnote_text = re.sub(r"  ", " ", headnote_text)
    judgment_text = re.sub(r"\n", " ", judgment_text)
    while "  " in judgment_text:
        judgment_text = re.sub(r"  ", " ", judgment_text)
    data['headnote'] = headnote_text.strip()
    data['judgment'] = judgment_text.strip()
    return data


from io import StringIO
train_data =[]
test_data=[]
val_data=[]
count = 0
for data_item in judgement_files:
    file_path = '/content/English-2000-Judgments/' + str(data_item)
    with open(file_path,'r') as f:
        text = f.read()

    extracted_data = preprocess(text)
    if data_item in test_file_names:
      test_data.append(extracted_data)
    elif data_item in val_file_names:
      val_data.append(extracted_data)
    else:
      train_data.append(extracted_data)

    count += 1

2224


In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)
val_dataset = Dataset.from_list(val_data)

In [ ]:
import tensorflow as tf
from transformers import TFMT5ForConditionalGeneration, MT5Tokenizer, DataCollatorForSeq2Seq
tokenizer=MT5Tokenizer.from_pretrained('google/mt5-small')
model=TFMT5ForConditionalGeneration.from_pretrained('google/mt5-small')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


tf_model.h5:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(new_data):
    padding="max_length"
    max_length=512
    inputs=[ex for ex in new_data['headnote']]
    targets=[ex for ex in new_data['judgment']]
    model_inputs = tokenizer(inputs, max_length=max_length, padding=padding, truncation=True)
    labels = tokenizer(targets, max_length=max_length, padding=padding, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
train_dataset = train_dataset.map(preprocess_function, batched=True, desc="Running tokenizer")
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=tokenizer.pad_token_id,
    pad_to_multiple_of=64,
    return_tensors="np")

tf_train_dataset = model.prepare_tf_dataset(
    train_dataset,
    collate_fn=data_collator,
    batch_size=4,
    )

Running tokenizer:   0%|          | 0/2204 [00:00<?, ? examples/s]

In [ ]:
model.compile(optimizer='Adam')
model.fit(tf_train_dataset, epochs=5)

Epoch 1/5


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
551/551 [==============================] - 354s 474ms/step - loss: 5.1489
Epoch 2/5
551/551 [==============================] - 256s 465ms/step - loss: 3.4766
Epoch 3/5
551/551 [==============================] - 255s 464ms/step - loss: 3.0872
Epoch 4/5
551/551 [==============================] - 255s 463ms/step - loss: 2.8579
Epoch 5/5
551/551 [==============================] - 255s 463ms/step - loss: 2.6676


In [ ]:
model.save_pretrained("finetuned_model")

In [ ]:
val_sample=val_dataset[0]
val_sample_inputs=tokenizer(val_sample['judgment'],return_tensors='tf',max_length=512, padding="max_length", truncation=True)

In [ ]:
token_ids = val_sample_inputs['input_ids'].numpy().tolist()[0]
val_sample_text = tokenizer.decode(token_ids)

In [ ]:
token_ids_tensor = tf.constant([token_ids])
generated_ids = model.generate(token_ids_tensor,max_new_tokens=500)
decoded_output = [tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]

In [ ]:
val_sample_text

'CIVIL APPELLATE JURISDICTION; Civil Appeal No. 698 of 1957. Appeal by special leave from the judgment and order dated August 2, 1956, of the Bombay High Court in Income-tax Reference No. 1 of 1956. A. V. Viswanatha Sastri, B. A. Palkhiwala and G. Gopalakrishnan, for the appellant. Hardyal Hardy and D. Gupta, for the respondent. 1960. December 5. The Judgment of the Court was delivered by SHAH, J.-The High Court of Judicature at Bombay answered the following two questions referred by the Income Tax Appellate Tribunal, Bench "B", Bombay, under s. 66(1) of the Indian Income-Tax Act, 1922: (1) Whether the acquisition of the managing agency 906 of the Dawn Mills Co., Ltd., was in the nature of a it n "business" carried on by the assessee company? (2)..If the answer to the first question is in the affirmative, whether the loss suffered by the assessee of...company of Rs. 1,78,438 on purchase and sale of 400 shares of the Dawn Mills Co., Ltd., being incidental to its business of acquiring th

In [ ]:
decoded_output

['CIVIL APPELLATE JURISDICTION: Civil Appeal No. 1564 of 1966. Appeal by special leave from the judgment and order dated October 3, 1966 of the Bombay High Court in Income-tax Reference No. 1564 of 1966. Appeal by special leave from the judgment and order dated October 3, 1966 of the Bombay High Court in Income-tax Reference No. 1564 of 1966. Appeal by special leave from the judgment and order dated October 3, 1966 of the Bombay High Court in Income-tax Reference No. 1564 of 1966. Appeal by special leave from the judgment and order dated October 3, 1966 of the Bombay High Court in Income-tax Reference No. 1564 of 1966. Appeal No. 1564 of 1966. Appeal by special leave from the judgment and order dated October 3, 1966 of the Bombay High Court in Income-tax Reference No. 1564 of 1966. Appeal No. 1564 of 1966. Appeal by special leave from the judgment and order dated October 3, 1966 of the Bombay High Court in Income-tax Reference No. 1564 of 1966. Appeal No. 1564 of 1966. Appeal No. 1564 

In [ ]:
from rouge_score import rouge_scorer

def calculate_rouge(reference, hypothesis):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, hypothesis)
    return scores
reference = val_sample_text
hypothesis = decoded_output[0]
rouge_scores = calculate_rouge(reference, hypothesis)
print(rouge_scores)

{'rouge1': Score(precision=0.3074433656957929, recall=0.30158730158730157, fmeasure=0.30448717948717946), 'rouge2': Score(precision=0.11688311688311688, recall=0.11464968152866242, fmeasure=0.1157556270096463), 'rougeL': Score(precision=0.23300970873786409, recall=0.22857142857142856, fmeasure=0.23076923076923078)}


In [ ]:
val_sample=val_dataset[1]
val_sample_inputs=tokenizer(val_sample['judgment'],return_tensors='tf',max_length=512, padding="max_length", truncation=True)
token_ids = val_sample_inputs['input_ids'].numpy().tolist()[0]
val_sample_text = tokenizer.decode(token_ids)
token_ids_tensor = tf.constant([token_ids])
generated_ids = model.generate(token_ids_tensor,max_new_tokens=500)
decoded_output = [tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]

In [ ]:
val_sample_text

'CRIMINAL APPELLATE JURISDICTION. Criminal Appeal No. 106 of 1956. Appeal from the Judgment and Order dated the 7th April, 1956, of the former Judicial Commissioner’s 65 510 Court, Vindhya Pradesh, Rewa in Misc. Crl. Application No. 70 of 1956. A. D. Mathur for the Appellant. B. K. B. Naidu and I. N. Shroff for the Respondent. 1960. November 18. The Judgment of the Court was delivered by IMAM, J.-The Judicial Commissioner of Vindhya Pradesh granted a certificate under Art. 132(1) of the Constitution of India as in his opinion the case involved a substantial question of law as to the interpretation of the Constitution. Hence the present appeal. The appellant was convicted under s. 307, Indian Penal Code and s. 19(f) of the Indian Arms Act by the Sessions Judge of Chatarpur. He was sentenced to 10 years’ rigorous imprisonment under s. 307, Indian Penal Code and to 3 years’ rigorous imprisonment under s. 19(f) of the Indian Arms Act. He filed an appeal while he was in jail which was summa

In [ ]:
decoded_output

['CIVIL APPELLATE JURISDICTION: Civil Appeal No. 88 of 1957. Appeal by special leave from the judgment and order dated October 28, 1955, of the former Judicial Commissioner, Delhi, in Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal by special leave from the judgment and order dated October 28, 1955, of the former Judicial Commissioner, Delhi, in Appeal No. 88 of 1955. Appeal by special leave from the judgment and order dated October 28, 1955, of the former Judicial Commissioner, Delhi, in Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal by special leave from the judgment and order dated October 28, 1955, of the former Judicial Commissioner, Delhi, in Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal by special leave from the judgment and order dated October 28, 1955, of the Additional Sessions Judge, Delhi, in Criminal Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 19

In [ ]:
reference = val_sample_text
hypothesis = decoded_output[0]
rouge_scores = calculate_rouge(reference, hypothesis)
print(rouge_scores)

{'rouge1': Score(precision=0.3090277777777778, recall=0.27469135802469136, fmeasure=0.29084967320261434), 'rouge2': Score(precision=0.10452961672473868, recall=0.09287925696594428, fmeasure=0.09836065573770493), 'rougeL': Score(precision=0.22916666666666666, recall=0.2037037037037037, fmeasure=0.2156862745098039)}


In [ ]:
val_sample=val_dataset[2]
val_sample_inputs=tokenizer(val_sample['judgment'],return_tensors='tf',max_length=512, padding="max_length", truncation=True)
token_ids = val_sample_inputs['input_ids'].numpy().tolist()[0]
val_sample_text = tokenizer.decode(token_ids)
token_ids_tensor = tf.constant([token_ids])
generated_ids = model.generate(token_ids_tensor,max_new_tokens=500)
decoded_output = [tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]

In [ ]:
val_sample_text

'CIVIL APPELLATE JURISDICTION: Civil Appeal No. 761 of 1957. Appeal by special leave from the judgment and order dated February 24, 1955, of the former Bombay High Court in I.T.R. 48/X of 1954. Hardayal Hardy and D. Gupta, for the appellant. N. A. Palkhivala and I. N. Shroff, for the respondent. 1960. November 17. The Judgment of the Court was delivered by SHAH, J.-The Income Tax Appellate Tribunal, Bombay Bench "A", referred under s. 66(1) of the Indian Income Tax Act, 1922-hereinafter referred to as the Act-the following question: "Whether the sum of Rs. 15,608 should have been included in the assessee Company’s "profit" for the purpose of determining whether the payment of a larger dividend than that declared by it would be unreasonable?" The High Court answered the question in the negative. Against the order of the High Court, with special leave under Art. 136 of the Constitution, this appeal is preferred. M/s. Bipinchandra Maganlal & Co., Ltd.-hereinafter referred to as the Compan

In [ ]:
decoded_output

['CIVIL APPELLATE JURISDICTION: Civil Appeal No. 1564 of 1966. Appeal by special leave from the judgment and order dated February 24, 1966 of the Bombay High Court in Income-tax Reference No. 1564 of 1966. Appeal by special leave from the judgment and order dated February 24, 1966 of the Bombay High Court in Income-tax Reference No. 1564 of 1966. M. C. Chagla and S. P. Nayar, for the appellant. M. C. Chagla and S. P. Nayar, for the respondent. The Judgment of the Court was delivered by Shah, J. This appeal by special leave is directed against the judgment of the High Court of Bombay, dated February 24, 1966. The appellant is the respondent. The appellant is the respondent. The appellant is the respondent. The appellant is the respondent. The appellant is the respondent. The appellant is the respondent. The appellant is the respondent. The appellant is the respondent. The appellant is the respondent. The appellant is the respondent. The appellant is the respondent. The appellant is the 

In [ ]:
reference = val_sample_text
hypothesis = decoded_output[0]
rouge_scores = calculate_rouge(reference, hypothesis)
print(rouge_scores)

{'rouge1': Score(precision=0.36012861736334406, recall=0.3333333333333333, fmeasure=0.3462132921174652), 'rouge2': Score(precision=0.13870967741935483, recall=0.12835820895522387, fmeasure=0.13333333333333333), 'rougeL': Score(precision=0.26366559485530544, recall=0.24404761904761904, fmeasure=0.2534775888717156)}


In [ ]:
val_sample=val_dataset[3]
val_sample_inputs=tokenizer(val_sample['judgment'],return_tensors='tf',max_length=512, padding="max_length", truncation=True)
token_ids = val_sample_inputs['input_ids'].numpy().tolist()[0]
val_sample_text = tokenizer.decode(token_ids)
token_ids_tensor = tf.constant([token_ids])
generated_ids = model.generate(token_ids_tensor,max_new_tokens=500)
decoded_output = [tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]

In [ ]:
val_sample

{'headnote': 'The respondent was a sub-Inspector of Police. A complaint was received by the Superintendent of Police that the com- plainant was carrying currency notes of Rs. 650 in a bundle when he was stopped by the respondent and his person was searched, that the respondent opened the bundle of notes and handed over the notes one by one to one Lalji, who was with him and that Lalji returned the notes to him but on reaching home he found the notes short by Rs. 250. Proceedings under s. 7 of the Police Act were taken against the respondent on the charge of misappropriation of Rs. 250 and he was dismissed from service by an order of the Deputy Inspector General of Police. The respondent filed a writ petition before the High Court challenging the order of the dismissal on the ground that the authorities had acted in violation of Rule I of Para. 486 of the U. P. Police Regulation. This rule required that every information received by the police relating to the commission of a cognizable 

In [ ]:
decoded_output

['CIVIL APPELLATE JURISDICTION: Civil Appeal No. 1564 of 1966. Appeal by special leave from the judgment and order dated January 9, 1963 of the Allahabad High Court in Civil Misc. Case No. 1564 of 1966. Appeal by special leave from the judgment and order dated January 9, 1963 of the Allahabad High Court in Civil Misc. Case No. 1564 of 1966. M. C. Chagla and S. P. Nayar, for the appellants. A. V. Gupte, S. P. Nayar, for the respondents. The Judgment of the Court was delivered by Shah, J. This appeal by special leave is directed against the judgment of the Allahabad High Court in Civil Misc. Case No. 1564 of 1966. The respondents were tried to carry on business at the respondents. The respondents were carrying on business at the respondents. The respondents were carrying on business at the respondents. The respondents were carrying on business at the respondents. The respondents were carrying on business at the respondents. The respondents were carrying on business at the respondents. Th

In [ ]:
reference = val_sample_text
hypothesis = decoded_output[0]
rouge_scores = calculate_rouge(reference, hypothesis)
print(rouge_scores)

{'rouge1': Score(precision=0.3559322033898305, recall=0.3387096774193548, fmeasure=0.34710743801652894), 'rouge2': Score(precision=0.16326530612244897, recall=0.1553398058252427, fmeasure=0.15920398009950246), 'rougeL': Score(precision=0.3050847457627119, recall=0.2903225806451613, fmeasure=0.2975206611570248)}


In [ ]:
val_sample=val_dataset[4]
val_sample_inputs=tokenizer(val_sample['judgment'],return_tensors='tf',max_length=512, padding="max_length", truncation=True)
token_ids = val_sample_inputs['input_ids'].numpy().tolist()[0]
val_sample_text = tokenizer.decode(token_ids)
token_ids_tensor = tf.constant([token_ids])
generated_ids = model.generate(token_ids_tensor,max_new_tokens=500)
decoded_output = [tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]

In [ ]:
val_sample_text

'CIVIL APPELLATE JURISDICTION: Civil Appeal No. 311 of 1959. Appeal from the judgment and order dated October 31, 1955, of the Travancore Cochin High Court, Ernakulam, in Original Petition No. 75 of 1955. A. N. Kripal and D. Gupta, for the appellant. Sardar Bahadur, for the respondent. 1960. December 13. The Judgment of the Court was delivered by 238 KAPUR, J.-This is an appeal pursuant to a certificate of the High Court of Kerala against the judgment and order of that court and the question for decision is the applicability of s. 35 of the Indian Income-tax Act (hereinafter termed the ’Act’). The facts which have given rise to the appeal are these: The respondent is a limited company which owns a spinning mills at Alwaye. It commenced business in January, 1951, and its first accounting year ended on December 31, 1951, and the relevant assessment year is 1952-53. It filed its return showing an income Rs. 3,21,284 without taking into account the amount allowable under s. 15C of the Act.

In [ ]:
decoded_output

['CIVIL APPELLATE JURISDICTION: Civil Appeal No. 88 of 1957. Appeal by special leave from the judgment and order dated January 25, 1955, of the Kerala High Court in Income-tax Reference No. 88 of 1955. Appeal from the judgment and order dated January 25, 1955, of the Kerala High Court in Income-tax Reference No. 88 of 1955. Appeal from the judgment and order dated January 25, 1955, of the High Court of Judicature at Madras in Income-tax Reference No. 88 of 1955. Appeal No. 88 of 1955. Appeal by special leave from the judgment and order dated January 25, 1955, of the Income-tax Appellate Tribunal, Madras, in Income-tax Reference No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 1955. Appeal No. 88 of 1955. App

In [ ]:
reference = val_sample_text
hypothesis = decoded_output[0]
rouge_scores = calculate_rouge(reference, hypothesis)
print(rouge_scores)

{'rouge1': Score(precision=0.28620689655172415, recall=0.25227963525835867, fmeasure=0.2681744749596123), 'rouge2': Score(precision=0.11764705882352941, recall=0.10365853658536585, fmeasure=0.11021069692058345), 'rougeL': Score(precision=0.1896551724137931, recall=0.16717325227963525, fmeasure=0.17770597738287558)}


In [ ]:
val_sample=val_dataset[5]
val_sample_inputs=tokenizer(val_sample['judgment'],return_tensors='tf',max_length=512, padding="max_length", truncation=True)
token_ids = val_sample_inputs['input_ids'].numpy().tolist()[0]
val_sample_text = tokenizer.decode(token_ids)
token_ids_tensor = tf.constant([token_ids])
generated_ids = model.generate(token_ids_tensor,max_new_tokens=500)
decoded_output = [tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]

In [ ]:
val_sample_text

'CIVIL APPELLATE JURISDICTION: Civil Appeals Nos. 98 and 99 of 1957. Appeals from the judgment and order dated August 31, 1954, of the Madhya Pradesh High Court in Civil Misc. Case No. 9 of 1953. R. Ganapathi Iyer and D. Gupta, for the appellant in C. A. No. 98 of 1957 and respondents in C. A. No. 99 of 1957. S. K. Kapur and Naunit Lal, for the respondents in C. A. No. 98 of 1957 and appellant in C. k. No. 99 of 1957. 1960. November 30. The Judgment of the Court was delivered by. MUDHOLKAR, J.-These are cross appeals from two judgments of the erstwhile High Court of Madhya Bharat. Both of them arise out of a writ petition presented by the Gwalior Sugar Company Ltd., who are respondents in C. A. 98 of 1957, in which they challenged the validity of the levy of a cess on sugarcane purchased by the respondents. The grounds on which the validity of the cess is challenged are two. The first ground is that it was not levied under any law and the second ground is that it is discriminatory agai

In [ ]:
decoded_output

['CIVIL APPELLATE JURISDICTION: Civil Appeal No. 88 of 1957. Appeal by special leave from the judgment and order dated July 23, 1957, of the Madhya Pradesh High Court in C. Appeal No. 88 of 1957. M. C. Setalvad, Attorney-General for India (G. N. Joshi and S. P. Nayar, for the appellant. M. C. Setalvad, Attorney-General for India (G. N. Joshi and S. P. Nayar, for the respondents. The Judgment of the Court was delivered by Shah, J. This appeal by special leave is directed against the respondents. The appellant is the respondents in C. A. No. 88 of 1957. The appellant is the respondents in C. A. No. 88 of 1957. The appellant is the respondents in C. A. No. 88 of 1957. The appellant is the respondents in C. A. No. 88 of 1957. The appellant is the respondents in C. A. No. 88 of 1957. The appellant is the respondents in C. A. No. 88 of 1957. The appellant is the respondents in C. A. No. 88 of 1957. The appellant is the respondents in C. A. No. 88 of 1957. The appellant is the respondents in 

In [ ]:
reference = val_sample_text
hypothesis = decoded_output[0]
rouge_scores = calculate_rouge(reference, hypothesis)
print(rouge_scores)

{'rouge1': Score(precision=0.4880546075085324, recall=0.41690962099125367, fmeasure=0.449685534591195), 'rouge2': Score(precision=0.18835616438356165, recall=0.1608187134502924, fmeasure=0.17350157728706625), 'rougeL': Score(precision=0.3515358361774744, recall=0.30029154518950435, fmeasure=0.3238993710691823)}
